# Combine IDOC population data sets

IDOC publishes population data sets which consist of each person housed in a state prison each quarter and information about them, such as their demographics, the type of crime they're being held for, the date on which they were apprehended, etc. The data sets are .xls files, but the titles, URLs, and data within each data set differ and need to be cleaned. This script combines all population data sets and makes the information within them consistent.

The population data sets can be found here: https://www2.illinois.gov/idoc/reportsandstatistics/Pages/Prison-Population-Data-Sets.aspx

This was last updated with the Dec 2021 data set as the most recently published data set.

In [1]:
import calendar
import warnings
from datetime import datetime as dt, date, timedelta as td

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import requests
from bs4 import BeautifulSoup
import xlrd

%matplotlib inline
inline_rc = dict(mpl.rcParams)

pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# helper function to get list of hex codes from tableau 20 pallette
def get_hex(n):
    cmap = cm.get_cmap('tab20', n)
    return [mcolors.rgb2hex(cmap(i)[:3]) for i in range(cmap.N)]

# helper function to get rgb code from hex code
def get_rgb_from_hex(hex_code):
    hex_code = hex_code.lstrip('#')
    return tuple(int(hex_code[i:i+2], 16) for i in (0, 2, 4))


Get links to the population data sets from the IDOC site.

In [ ]:
#updated February 6, 2026 to account for the new format of the links on teh IDOC website.
# The links are now in the format of "https://idoc.illinois.gov/content/dam/soi/en/web/idoc/reportsandstatistics/prison-population-data-sets/2025/2025-01.xlsx" instead of "https://idoc.illinois.gov/reportsandstatistics/prison-population-data-sets/2025-01.xlsx". The code now uses the urljoin function to join the base URL with the relative URL of the Excel files.
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import pandas as pd

PAGE_URL = "https://idoc.illinois.gov/reportsandstatistics/prison-population-data-sets.html"

response = requests.get(PAGE_URL, timeout=60)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

files = []

date_pattern = re.compile(r'on (\d{2}-\d{2}-\d{2})')

for a in soup.select("a[href]"):
    label = a.get_text(" ", strip=True)
    href = a["href"]

    if re.search(r"\.(xls|xlsx)$", href, re.I):

        match = date_pattern.search(label)

        if match:
            doc_date = pd.to_datetime(match.group(1), format="%m-%d-%y")

            files.append({
                "date": doc_date,
                "url": urljoin(PAGE_URL, href),
                "label": label
            })

print(f"Found {len(files)} files")


Read in and combine the data sets. This takes a bit.

In [ ]:
from io import BytesIO
import requests

def read_idoc_excel(url):
    
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    
    raw = pd.read_excel(BytesIO(r.content), header=None, engine="xlrd")

    # find header row dynamically
    header_row = None
    for i in range(40):
        if (raw.iloc[i].astype(str).str.contains("IDOC")).any():
            header_row = i
            break

    if header_row is None:
        raise ValueError(f"Could not find header in {url}")

    df = pd.read_excel(
        BytesIO(r.content),
        skiprows=header_row,
        dtype="object",
        engine="xlrd"
    )

    # drop garbage unnamed cols
    df = df.loc[:, ~df.columns.astype(str).str.contains("^Unnamed")]

    return df


    

In [ ]:
df_list = []

for i, file in enumerate(files, 1):

    print(f"{i}/{len(files)} — {file['date'].date()}")

    df = read_idoc_excel(file["url"])

    df["Doc Date"] = file["date"]

    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)

Export file so I don't have to read these in every time.

In [ ]:
# df.to_csv('raw_full_pop_data.csv')

# df = pd.read_csv('raw_full_pop_data.csv', dtype=object)


Remove notes at the end of each doc

In [ ]:
# --- Normalize column names immediately after reading ---
df.columns = (
    df.columns
        .str.strip()
        .str.lower()
        .str.replace(r"\s+", "_", regex=True)
)

# --- Ensure expected column exists ---
if "name" not in df.columns:
    raise ValueError("Expected column 'name' not found — schema may have changed.")

# --- Drop blank rows safely ---
df = df.loc[df["name"].notna()].reset_index(drop=True)

# --- Optional: create a stable row id (ONLY if you truly need it) ---
df.insert(0, "row_id", range(len(df)))

**Explore the data**

In [ ]:
print(df.info())



**Dedupe/combine columns**

Sometimes, columns in the individual data sets had different names. Combine these into a single column in the combined version.


In [ ]:
import re
import pandas as pd

def normalize_cols(cols):
    """Normalize column names to reduce accidental mismatches."""
    return (
        pd.Index(cols)
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
    )

def consolidate_duplicate_columns(
    df: pd.DataFrame,
    base_names: list[str],
    max_suffix: int = 5,
    rename_map: dict[str, str] | None = None,
    drop_all_null_cols: bool = True,
    verbose: bool = True,
) -> pd.DataFrame:
    """
    For each base name, fill nulls in the base column from any suffixed variants:
      Base, Base1, Base2, ... Base{max_suffix-1}
    Then drop the suffixed columns.
    """
    df = df.copy()

    # 1) normalize existing column names (whitespace etc.)
    df.columns = normalize_cols(df.columns)

    # 2) optional renames to align known variants
    if rename_map:
        rename_map = {k.strip(): v.strip() for k, v in rename_map.items()}
        df = df.rename(columns=rename_map)

    # 3) consolidate
    for base in base_names:
        base = base.strip()
        if base not in df.columns:
            if verbose:
                print(f"\n[skip] Base column not found: {base!r}")
            continue

        if verbose:
            print(f"\nCombining dupes for: {base}")
            print("  start non-null:", df[base].notna().sum())

        # find all suffixed columns matching base + digits (e.g., "Admission Type1")
        # we include "base" itself separately; here we only collect suffix variants
        pattern = re.compile(rf"^{re.escape(base)}(\d+)$")
        suffix_cols = [c for c in df.columns if pattern.match(c)]

        # optionally also look for accidental double-space variants already normalized;
        # normalization should handle most of these.

        # Fill base from each suffix col in order of suffix number
        def suffix_num(c):
            return int(pattern.match(c).group(1))

        for c in sorted(suffix_cols, key=suffix_num):
            before = df[base].notna().sum()
            df[base] = df[base].fillna(df[c])
            after = df[base].notna().sum()
            if verbose:
                print(f"  filled from {c}: {before} -> {after}")
            df = df.drop(columns=[c])

    # 4) drop fully empty columns
    if drop_all_null_cols:
        empty_cols = [c for c in df.columns if df[c].isna().all()]
        if verbose:
            print("\nDropping empty columns:")
            for c in empty_cols:
                print(" ", c)
        if empty_cols:
            df = df.drop(columns=empty_cols)

    return df
def to_snake(s: str) -> str:
    return (
        str(s)
        .strip()
        .lower()
        .replace("  ", " ")
        .replace(" ", "_")
    )

base_names = [
    "Projected Mandatory Supervised Release (MSR) Date",
    "Projected Discharge Date",
    "Current Admission Date",
    "Admission Type",
]
base_names = [to_snake(x) for x in base_names]

rename_map = {
    "Projected Discharge  Date2": "Projected Discharge Date1",
    "Current Admission Type": "Admission Type1",
}
# normalize rename_map keys/values too
rename_map = {to_snake(k): to_snake(v) for k, v in rename_map.items()}

df = consolidate_duplicate_columns(
    df,
    base_names=base_names,
    rename_map=rename_map,
    verbose=True,
)




**Explore further**

Which fields need cleaning?

In [ ]:
# drop accidental index column if it exists
if "index" in df.columns:
    df = df.drop(columns=["index"])

# if you don't need both:
# keep row_id, drop idx (or vice versa)
if "idx" in df.columns:
    df = df.drop(columns=["idx"])

In [ ]:
for col in df.columns:
    print('\n')
    print(df[col].value_counts(dropna=False))
    

* Sex has nulls and incorrect values
* Race has nulls and values that can be combined
* Veteran Status has values that can be combined
* Parent Institution has values that can be combined
* Crime Class has nulls and values that can be combined
* Sentencing County has values that can be combined and should be categorized


* Truth in Sentencing needs % split into its own column


* Sentence Years needs further exploration - includes 0002, 999, LFE
* Sentence Months needs further exploration - do these match Sentence Years?


* Date of Birth has multiple date formats
* Current Admission Date has multiple date formats
* Custody Date has multiple date formats
* Sentence Date has multiple date formats
* Projected Mandatory Supervised Release (MSR) Date has multiple date formats and non-date values
* Projected Discharge Date has multiple date formats and non-date values


* Admission Type needs further exploration
* Holding Offense needs further exploration and should be categorized
    * Holding Offense Category exists, but use Phil's categories as well
    * Offense Type exists, but use Phil's categories as well


**Clean Sex**

Sex contains empty spaces and "B" value.

In [ ]:
# standardize the column first
df["sex"] = (
    df["sex"]
        .astype(str)
        .str.strip()
        .replace({
            "B": "Unknown",   # bad code sometimes appears
            "": "Unknown",
            " ": "Unknown",
            "nan": "Unknown"
        })
        .fillna("Unknown")
)

print(df["sex"].value_counts(dropna=False))


**Clean Race**

Race contains empty spaces, Unknown, and Not Assigned.

In [ ]:
race_map = {
    "W": "White",
    "B": "Black",
    "H": "Hispanic",
    "A": "Asian",
    "I": "Indigenous",
    "O": "Other",
}

df["race"] = (
    df["race"]
      .astype(str)
      .str.strip()
      .replace({
          "": "Unknown",
          "Not Assigned": "Unknown",
          "Bi-Racial": "Other",
          "nan": "Unknown",
      })
)

# Final safety catch
print(sorted(df["race"].unique()))
print(df["race"].value_counts(dropna=False))






**Clean Veteran Status**

Veteran Status contains Yes, No, NaN, and Unknown. 

In [ ]:
# check if NaN means No or Unknown
df["veteran_status"] = (
    df["veteran_status"]
        .astype(str)
        .str.strip()
        .replace({
            "": pd.NA,
            "nan": pd.NA
        })
)

# plenty of Nos in the same documents as the NaNs, so NaN likely means "Unknown"
df["veteran_status"] = df["veteran_status"].fillna("Unknown")

status_map = {
    "Yes": "Yes",
    "No": "No",
    "Unknown": "Unknown"
}

df["veteran_status"] = (
    df["veteran_status"]
        .map(status_map)
        .fillna("Unknown")
)

df["veteran_status"] = df["veteran_status"].astype("category")

**Parent Intitutions**

Clean up the names to be consistent.

In [ ]:
# remove CC appendix from  names
df["parent_institution"] = (
    df["parent_institution"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s*CC$", "", regex=True)
)

df["parent_institution"] = df["parent_institution"].replace({
    "Southwestern IL": "Southwestern Illinois"
})


df["parent_institution"] = df["parent_institution"].astype("category")

In [ ]:
#double checking -- to see if cleaning and standardization worked as expected and to check for any remaining anomalies in the parent_institution column.
sorted(df["parent_institution"].unique())

In [ ]:
df.nunique().sort_values(ascending=False).head(15)

In [ ]:
#standardize dates 

date_cols = [
    "date_of_birth",
    "custody_date",
    "sentence_date",
    "current_admission_date",
    "projected_mandatory_supervised_release_(msr)_date",
    "projected_discharge_date",
    "doc_date",
]

for c in date_cols:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")


In [ ]:
#convert low cardinality columns to category dtype
cat_cols = ["sex", "race", "veteran_status", "admission_type", "sentencing_county", "parent_institution"]
for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype("category")

**Clean Crime Class**

* Class M is murder
* Change empty spaces, NaN, and Missing to Unknown
* Unclassified remains as Unclassified

In [ ]:
df["crime_class"] = (
    df["crime_class"]
        .astype(str)
        .str.strip()
        .replace({
            "Class M": "Murder",
            "": "Unknown",
            "Missing": "Unknown",
            "nan": "Unknown"
        })
        .fillna("Unknown")
        .astype("category")
)

print(df["crime_class"].value_counts(dropna=False))

#checking the unique values in crime_class to see if there are any remaining anomalies or if further standardization is needed.
#sorted(df["crime_class"].unique())

**Sentencing Counties**

* Clean up different versions of county names
* Categorize into regions

In [ ]:
import numpy as np
import pandas as pd

# --- 1) Standardize county strings ---
s = (
    df["sentencing_county"]
      .astype("string")          # keeps missing as <NA>
      .str.strip()
)

# Fix common variants (case-insensitive)
county_fixes = {
    "dekalb": "DeKalb",
    "dupage": "DuPage",
    "lasalle": "LaSalle",
    "out of state": "Out of state/Unknown",
    "unknown": "Out of state/Unknown",
}

# Apply fixes; if not in map, keep original
df["sentencing_county"] = s.str.lower().map(county_fixes).fillna(s)

# Optional: collapse blanks to unknown
df["sentencing_county"] = df["sentencing_county"].replace(["", " "], "Out of state/Unknown")

cook = {"COOK"}
chi_metro = {"DUPAGE", "LAKE", "KANE", "MCHENRY", "WILL"}
north_il = {"BOONE","BUREAU","CARROLL","DEKALB","GRUNDY","HENRY","JO DAVIESS","KENDALL","LASALLE","LEE","OGLE",
            "ROCK ISLAND","STEPHENSON","WHITESIDE","WINNEBAGO"}
central_il = {"ADAMS","BROWN","CALHOUN","CASS","CHAMPAIGN","CHRISTIAN","COLES","CUMBERLAND","DEWITT","DOUGLAS",
              "EDGAR","FORD","FULTON","GREENE","HANCOCK","HENDERSON","IROQUOIS","JERSEY","KANKAKEE","KNOX",
              "LIVINGSTON","LOGAN","MACON","MACOUPIN","MARSHALL","MASON","MCDONOUGH","MCLEAN","MENARD","MERCER",
              "MONTGOMERY","MORGAN","MOULTRIE","PEORIA","PIATT","PIKE","PUTNAM","SANGAMON","SCHUYLER","SCOTT",
              "SHELBY","STARK","TAZEWELL","VERMILION","WARREN","WOODFORD"}
st_louis = {"MADISON", "ST. CLAIR"}
south_il = {"ALEXANDER","BOND","CLARK","CLAY","CLINTON","CRAWFORD","EDWARDS","EFFINGHAM","FAYETTE","FRANKLIN",
            "GALLATIN","HAMILTON","HARDIN","JACKSON","JASPER","JEFFERSON","JOHNSON","LAWRENCE","MARION","MASSAC",
            "MONROE","PERRY","POPE","PULASKI","RANDOLPH","RICHLAND","SALINE","UNION","WABASH","WASHINGTON","WAYNE",
            "WHITE","WILLIAMSON"}

county_upper = df["sentencing_county"].astype("string").str.upper()

df["sentencing_region"] = pd.Categorical(
    np.select(
        [
            county_upper.isin(cook),
            county_upper.isin(chi_metro),
            county_upper.isin(north_il),
            county_upper.isin(central_il),
            county_upper.isin(st_louis),
            county_upper.isin(south_il),
        ],
        [
            "Cook",
            "Chicago Metro",
            "North IL",
            "Central IL",
            "St. Louis Metro",
            "South IL",
        ],
        default="Out of State/Unknown",
    )
)




In [ ]:
df.info(memory_usage="deep")

In [ ]:
# Canonical columns you expect in the final combined dataset
SCHEMA = {
    "idoc_#": "string",
    "name": "string",
    "date_of_birth": "datetime64[ns]",
    "sex": "category",
    "race": "category",
    "veteran_status": "category",
    "current_admission_date": "datetime64[ns]",
    "admission_type": "category",
    "parent_institution": "category",
    "projected_mandatory_supervised_release_(msr)_date": "datetime64[ns]",
    "projected_discharge_date": "datetime64[ns]",
    "custody_date": "datetime64[ns]",
    "sentence_date": "datetime64[ns]",
    "crime_class": "category",
    "holding_offense": "string",
    "holding_offense_category": "category",
    "offense_type": "category",
    "sentence_years": "Int64",
    "sentence_months": "Int64",
    "truth_in_sentencing": "category",
    "sentencing_county": "category",
    "sentencing_region": "category",
    "doc_date": "datetime64[ns]",
}
import pandas as pd
import numpy as np

def enforce_schema(df: pd.DataFrame, schema: dict, *, strict: bool = True) -> pd.DataFrame:
    df = df.copy()

    # 1) missing columns
    missing = [c for c in schema if c not in df.columns]
    if missing and strict:
        raise ValueError(f"Missing required columns: {missing}")
    for c in missing:
        df[c] = pd.NA

    # 2) unexpected columns (useful warning)
    extra = [c for c in df.columns if c not in schema]
    if extra and strict:
        # in strict mode, fail loudly (best for pipelines)
        raise ValueError(f"Unexpected columns found (schema drift): {extra}")

    # 3) cast types
    for col, dtype in schema.items():
        if col not in df.columns:
            continue

        if dtype.startswith("datetime"):
            df[col] = pd.to_datetime(df[col], errors="coerce")
        elif dtype == "Int64":
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        elif dtype == "string":
            df[col] = df[col].astype("string")
        elif dtype == "category":
            df[col] = df[col].astype("category")
        else:
            df[col] = df[col].astype(dtype)

    # 4) keep columns in canonical order
    df = df[list(schema.keys())]
    return df
df["doc_date"] = pd.to_datetime(df["doc_date"], errors="coerce")

# enforce schema per-file
df = enforce_schema(df, SCHEMA, strict=False)   # start with strict=False while you’re still harmonizing

df_list.append(df)

In [ ]:
df = enforce_schema(df, SCHEMA, strict=True)

In [ ]:
def schema_report(df: pd.DataFrame, schema: dict) -> None:
    expected = set(schema.keys())
    got = set(df.columns)
    missing = sorted(expected - got)
    extra = sorted(got - expected)
    if missing: print("Missing:", missing)
    if extra: print("Extra:", extra)



In [ ]:
schema_report(df, SCHEMA)

**Truth In Sentencing**

Clean up the mix of floats and strings, percents and descriptions.

In [ ]:
tis = (
    df["truth_in_sentencing"]
        .astype("string")
        .str.strip()
        .replace({"": pd.NA, "nan": pd.NA, "Missing": pd.NA})
)

pct = tis.str.extract(r'(\d+\.?\d*)\s*%')[0]
df["tis_pct"] = pd.to_numeric(pct, errors="coerce") / 100

decimal_mask = tis.str.match(r'^0?\.\d+$|^1\.0+$|^1$')
df.loc[decimal_mask, "tis_pct"] = pd.to_numeric(tis[decimal_mask], errors="coerce")

df.loc[tis.str.contains("day", case=False, na=False), "tis_pct"] = 0.5

df["tis_desc"] = (
    tis
      .str.replace(r'^\d+\.?\d*\s*%', '', regex=True)
      .str.strip()
)

df.loc[df["tis_desc"] == "", "tis_desc"] = pd.NA

life_mask = df["sentence_years"].isin(["LIFE", "SDP"])
df.loc[life_mask, ["tis_pct", "tis_desc"]] = pd.NA

df["tis_pct"] = df["tis_pct"].astype("float32")
df["tis_desc"] = df["tis_desc"].astype("category")



In [ ]:
df["tis_pct"].value_counts().head(10)
df["truth_in_sentencing"] = tis.astype("category")
df.info(memory_usage="deep")

**Sentence Length**

* The formatting of the sentence years and months is inconsistent.
* Create columns showing the total sentencing length in months and in years.

In [ ]:
import numpy as np
import pandas as pd

SPECIAL = {"LIFE", "SDP", "DEATH", "Unknown"}

# --- normalize raw strings (years + months) ---
def _norm_series(s: pd.Series) -> pd.Series:
    return (
        s.astype("string")
         .str.strip()
         .replace({"": pd.NA, "nan": pd.NA, " ": pd.NA})
    )

df["sentence_years_raw"] = _norm_series(df["sentence_years"])
df["sentence_months_raw"] = _norm_series(df["sentence_months"])

# years: fix typos + unknowns
df["sentence_years_raw"] = df["sentence_years_raw"].replace({
    "LFE": "LIFE",
    "PEND": "Unknown",
})
df["sentence_years_raw"] = df["sentence_years_raw"].fillna("Unknown")

# months: if years is special, force months to same special label
is_special_year = df["sentence_years_raw"].isin(SPECIAL)
df.loc[is_special_year, "sentence_months_raw"] = df.loc[is_special_year, "sentence_years_raw"]

# months: unknown handling for non-special
df.loc[~is_special_year, "sentence_months_raw"] = df.loc[~is_special_year, "sentence_months_raw"].fillna("Unknown")


# --- parse numeric years/months where possible ---
years_num = pd.to_numeric(df["sentence_years_raw"], errors="coerce")
months_num = pd.to_numeric(df["sentence_months_raw"], errors="coerce")

# where years are numeric AND months numeric, keep them
is_numeric = years_num.notna() & months_num.notna()

# fix cases:
# (A) if years > 0 and months >= 12 -> set months = 0 (trust years)
fix_months_gt12 = is_numeric & (years_num > 0) & (months_num >= 12)
months_num = months_num.mask(fix_months_gt12, 0)

# (B) if years == 0 and months >= 12 -> convert months to years+months
fix_years0_months_ge12 = is_numeric & (years_num == 0) & (months_num >= 12)
years_num = years_num.mask(fix_years0_months_ge12, (months_num // 12).astype("Int64"))
months_num = months_num.mask(fix_years0_months_ge12, (months_num % 12).astype("Int64"))

# store cleaned numeric versions (nullable ints)
df["sentence_years_int"] = years_num.round().astype("Int64")
df["sentence_months_int"] = months_num.round().astype("Int64")


# --- build analysis variables ---
# total months numeric (nullable)
df["sentence_total_months"] = (df["sentence_years_int"] * 12 + df["sentence_months_int"]).astype("Int64")

# continuous years numeric (float) where numeric
df["sentence_years_cont"] = (df["sentence_total_months"] / 12).astype("Float32")

# keep a clean label version for special sentences
df["sentence_years_label"] = df["sentence_years_raw"].where(df["sentence_years_raw"].isin(SPECIAL), pd.NA)
df["sentence_years_label"] = df["sentence_years_label"].astype("category")

# optional: drop raw helper cols after QA
# df = df.drop(columns=["sentence_years_raw", "sentence_months_raw"])


    


In [ ]:
print(df.loc[df["sentence_months_int"].notna(), "sentence_months_int"].describe())

mask = df["sentence_months_int"].notna() & (df["sentence_months_int"] >= 12)
print(df.loc[mask, ["sentence_years_int", "sentence_months_int"]].head())

print(df["sentence_years_label"].value_counts(dropna=False))
print("numeric rows:", df["sentence_total_months"].notna().sum(), "of", len(df))



**Reformat dates**

Some dates are saved as strings. These can be in various formats. They need to all be converted to dates.



In [ ]:
# what different formats can the dates take?
# (add len col and str col for each date col)

date_cols = [c for c in df.columns if "date" in c.lower()]

for col in date_cols:
    print("\n", col)

    lens = (
        df[col]
        .astype("string")
        .str.len()
        .value_counts(dropna=False)
        .sort_index()
    )

    print(lens)
date_cols = [
    "date_of_birth",
    "current_admission_date",
    "custody_date",
    "sentence_date",
    "projected_mandatory_supervised_release_(msr)_date",
    "projected_discharge_date",
    "doc_date",
]

for c in date_cols:
    # ensure datetime first, then cast to microseconds
    df[c] = pd.to_datetime(df[c], errors="coerce").values.astype("datetime64[us]")




In [ ]:
for c in date_cols:
    print(c, df[c].dtype)

In [ ]:
# play around with this code to see what the various lengths look like
dob_s = df["date_of_birth"].astype("string")
adm_s = df["current_admission_date"].astype("string")
cus_s = df["custody_date"].astype("string")
doc_s = df["doc_date"].astype("string")
id_s  = df["idoc_#"].astype("string")


#Show me examples where admission date string length == 10
mask = adm_s.str.len().eq(10)
print(df.loc[mask, ["current_admission_date"]].head(10))

#Check for a period in DOB where string length == 8
mask = dob_s.str.contains(r"\.", regex=True, na=False) & dob_s.str.len().eq(8)
print(df.loc[mask, ["date_of_birth", "idoc_#", "doc_date"]].head(10))


In [ ]:
#Show the actual unparsed / weird values

for col in ["date_of_birth", "current_admission_date", "custody_date", "sentence_date"]:
    s = df[col].astype("string").str.strip()
    parsed = pd.to_datetime(s, errors="coerce")

    bad = df.loc[parsed.isna() & s.notna(), col].astype("string").dropna().unique()[:20]
    print("\n", col, "examples of problematic values:")
    print(bad)


In [ ]:
# convert to dates
# (using None instead of np.nan so that string length can still be calculated)

import pandas as pd
import numpy as np

SENTINELS = {
    "00000000", "0 0 0 0", " 0000000",
    "99999999", "99959999", "99989999",
    "82220288",
}

def clean_date_column(s: pd.Series) -> pd.Series:
    """
    Clean a messy date column into pandas datetime (keeps out-of-bounds years by staying at us resolution).
    Handles:
      - ISO (YYYY-MM-DD or YYYY-MM-DD HH:MM:SS)
      - MMDDYYYY (8 digits)
      - MDDYYYY (7 digits)
      - trailing '.' and whitespace
      - sentinel values (00000000, 99999999, etc.)
      - very short junk (len <= 3)
    """
    # work as string for cleaning
    x = s.astype("string").str.strip()

    # blank / short junk -> NA
    x = x.mask(x.isna() | (x.str.len() <= 3), pd.NA)

    # strip a trailing period
    x = x.str.replace(r"\.$", "", regex=True)

    # null out any strings containing a period anywhere else (your old rule)
    x = x.mask(x.str.contains(r"\.", regex=True, na=False), pd.NA)

    # remove non-digit separators for numeric formats, but keep ISO intact
    # (we'll parse ISO separately first)
    iso = x.where(x.str.match(r"^\d{4}-\d{2}-\d{2}", na=False), pd.NA)
    non_iso = x.where(~x.str.match(r"^\d{4}-\d{2}-\d{2}", na=False), pd.NA)

    # sentinel codes (on the non-iso side)
    non_iso = non_iso.mask(non_iso.isin(SENTINELS), pd.NA)

    # keep only digits for numeric date formats (e.g. 01022020, 1022020)
    non_iso_digits = non_iso.str.replace(r"\D", "", regex=True)

    # 7-digit -> pad to 8 (MDDYYYY -> 0MDDYYYY)
    non_iso_digits = non_iso_digits.mask(non_iso_digits.str.len().eq(7), "0" + non_iso_digits)

    # Now parse:
    # ISO: just take first 10 chars as date
    parsed_iso = pd.to_datetime(iso.str.slice(0, 10), errors="coerce", format="%Y-%m-%d")

    # Numeric: MMDDYYYY
    parsed_num = pd.to_datetime(non_iso_digits, errors="coerce", format="%m%d%Y")

    # Combine: prefer ISO where available, else numeric
    out = parsed_iso.fillna(parsed_num)

    # IMPORTANT: keep microsecond resolution so far-future dates can exist (no ns casting)
    # Convert to numpy datetime64[us] array (safe)
    out = pd.to_datetime(out, errors="coerce").values.astype("datetime64[us]")

    return pd.Series(out, index=s.index)



In [ ]:
# convert the timestamps and dates to datetimes
date_cols = [c for c in df.columns if "date" in c.lower() and c != "doc_date"]

for c in date_cols:
    print("\nCleaning:", c)
    before_missing = df[c].isna().mean()

    df[c] = clean_date_column(df[c])

    after_missing = pd.isna(df[c]).mean()
    print("missing before:", round(before_missing, 4), "after:", round(after_missing, 4))
    print("dtype:", df[c].dtype)



**Missing/incorrect birth dates**

Some birth dates are missing or typoed. Replace with the mode.

In [ ]:
dob_conflicts = (
    df.groupby("idoc_#")["date_of_birth"]
      .nunique()
      .loc[lambda s: s > 1]
      .index
)

conflict_df = (
    df.loc[df["idoc_#"].isin(dob_conflicts), ["idoc_#", "date_of_birth"]]
      .drop_duplicates()
      .sort_values("idoc_#")
)

print(conflict_df)



In [ ]:
print(len(dob_conflicts) / df["idoc_#"].nunique())

(
    df.loc[df["idoc_#"].isin(dob_conflicts)]
      .groupby("idoc_#")["date_of_birth"]
      .agg(["min", "max", "nunique"])
      .head(20)
)


In [ ]:
# how many IDs have >1 DOB (before)
dob_nunique = df.groupby("idoc_#")["date_of_birth"].nunique(dropna=True)
print("Original conflicts:", (dob_nunique > 1).sum())
print("Original missing DOB rows:", df["date_of_birth"].isna().sum())

# compute DOB mode per person (ties broken deterministically by earliest date)
def dob_mode(s: pd.Series):
    s = s.dropna()
    if s.empty:
        return pd.NaT
    vc = s.value_counts()
    top = vc[vc == vc.max()].index
    return min(top)  # tie-break: smallest/earliest

dob_mode_map = df.groupby("idoc_#")["date_of_birth"].agg(dob_mode)

# overwrite DOB with per-person mode (preserves row count/order)
df["date_of_birth"] = df["idoc_#"].map(dob_mode_map)

# how many IDs have >1 DOB (after)
dob_nunique2 = df.groupby("idoc_#")["date_of_birth"].nunique(dropna=True)
print("\nNew conflicts:", (dob_nunique2 > 1).sum())
print("New missing DOB rows:", df["date_of_birth"].isna().sum())




**Add Date Diffs**
* Age
* Age at Custody
* Projected Time Until Release
* Projected Age at Release

(Note: Doing this in Tableau instead rounds to the nearest number, which produces unintuitive results.)

In [ ]:
# cap the projected dates at 2261; otherwise, element-wise operators won't work

NS_MAX = pd.Timestamp("2262-04-11")

for col in [
    "projected_discharge_date",
    "projected_mandatory_supervised_release_(msr)_date"
]:
    df[col + "_capped"] = df[col].clip(upper=NS_MAX)
for col in [
    "projected_discharge_date_capped",
    "projected_mandatory_supervised_release_(msr)_date_capped"
]:
    df[col] = df[col].astype("datetime64[ns]")



In [ ]:
# get age and age at custody as int

df["age"] = (
    (df["doc_date"] - df["date_of_birth"])
    .dt.days
    .floordiv(365.25)
    .astype("Int16")
)

df["age_at_custody"] = (
    (df["custody_date"] - df["date_of_birth"])
    .dt.days
    .floordiv(365.25)
    .astype("Int16")
)

print(df["age"].describe())


In [ ]:
#checking why there is an 8 and 9 year old
bad_age = df[(df["age"] < 10) | (df["age"] > 100)]

print(len(bad_age))
print(
    bad_age[
        ["idoc_#", "date_of_birth", "custody_date", "doc_date", "age", "age_at_custody", "crime_class", "holding_offense",
"holding_offense_category", "offense_type"]
    ]
)


In [ ]:
#checking for multiple crimes per person
multi_crime = (
    df.groupby("idoc_#")["holding_offense"]
      .nunique()
      .gt(1)
      .sum()
)

print(multi_crime)

In [ ]:
date_cols = [
    "date_of_birth",
    "current_admission_date",
    "custody_date",
    "sentence_date",
    "projected_mandatory_supervised_release_(msr)_date_capped",
    "projected_discharge_date_capped",
    "doc_date",
]

df3 = df.copy()
for c in date_cols:
    df3[c] = pd.to_datetime(df3[c], errors="coerce").dt.normalize().astype("datetime64[ns]")





In [ ]:
df3.to_parquet("idoc_clean_timestamps.parquet", index=False)
# df = df3.copy()

In [ ]:
msr_col = "projected_mandatory_supervised_release_(msr)_date_capped"
dis_col = "projected_discharge_date_capped"
doc_col = "doc_date"
dob_col = "date_of_birth"

# --- helper: coerce to day-level numpy datetime64[D] safely ---
def to_day_array(s: pd.Series) -> np.ndarray:
    # works for datetime64, python date objects, strings, etc.
    return pd.to_datetime(s, errors="coerce").to_numpy(dtype="datetime64[D]")

# 1) canonical projected release date (keep as-is; can be datetime or date objects)
df["projected_release_date_capped"] = df[msr_col]
mask = df[msr_col].isna() | (pd.to_datetime(df[msr_col], errors="coerce") <= pd.to_datetime(df[doc_col], errors="coerce"))
df.loc[mask, "projected_release_date_capped"] = df.loc[mask, dis_col]

# 2) compute age at release using day-resolution arrays (no .dt needed)
rel_D = to_day_array(df["projected_release_date_capped"])
dob_D = to_day_array(df[dob_col])

age_days = (rel_D - dob_D).astype("timedelta64[D]").astype("float64")
age_years = np.floor(age_days / 365.25)

df["projected_age_at_release_capped"] = pd.Series(age_years, index=df.index).astype("Float32").astype("Int16")

# if either date missing => NA
missing_age = pd.isna(pd.to_datetime(df["projected_release_date_capped"], errors="coerce")) | pd.isna(pd.to_datetime(df[dob_col], errors="coerce"))
df.loc[missing_age, "projected_age_at_release_capped"] = pd.NA

# 3) time until release (tenths of a year) using day-resolution arrays
doc_D = to_day_array(df[doc_col])
delta_days = (rel_D - doc_D).astype("timedelta64[D]").astype("float64")

x = 10 * (delta_days / 365.0)
x_adj = np.where(x < 0, x - 1, x)
df["projected_time_until_release_capped"] = (np.trunc(x_adj) / 10).astype("float32")

missing_time = pd.isna(pd.to_datetime(df["projected_release_date_capped"], errors="coerce")) | pd.isna(pd.to_datetime(df[doc_col], errors="coerce"))
df.loc[missing_time, "projected_time_until_release_capped"] = np.nan

# 4) quick QA
print(df["projected_age_at_release_capped"].describe())
print(df["projected_time_until_release_capped"].describe())


**Latest record**

Since the data is produced quarterly, some individuals will be repeated in the data. Get the date of the most recent document in which each individual appears.

Note: This takes a bit.

In [ ]:
len1 = len(df)
blank_date_df = df[pd.isnull(df['Current Admission Date'])]
non_blank_date_df = df[~pd.isnull(df['Current Admission Date'])]

non_blank_date_df['Current Admission Date str'] = non_blank_date_df['Current Admission Date'].apply(lambda x: dt.strftime(x, '%Y%m%d'))
non_blank_date_df['ID'] = non_blank_date_df['IDOC #'].add(non_blank_date_df['Current Admission Date str'])
non_blank_date_df.drop('Current Admission Date str', axis=1, inplace=True)

blank_date_df['ID'] = non_blank_date_df['IDOC #']
df = pd.concat([blank_date_df, non_blank_date_df])

if len(df) != len1:
    print(len(df))
    print(len1)
    print('blanking dates changed the length')
    1/0

# get most recent doc date
doc_date_series = df[['ID', 'Doc Date']]
doc_date_series.rename(columns={'Doc Date': 'Last Doc Date'}, inplace=True)
doc_date_series = doc_date_series.groupby('ID')['Last Doc Date'].max()
df = df.merge(doc_date_series, how='inner', on='ID')

# remove unecessary columns
for col in df.columns:
    if col[-4:] in [' len', ' str'] or col in ['index', 'Idx']:
        df.drop(col, axis=1, inplace=True)
# (need to drop Doc Date to remove all duplicates)
        
# remove duplicates
df.drop_duplicates(inplace=True)

# drop ID col
df.drop('ID', axis=1, inplace=True)



**Admission Type - Label Consistency**

Update labels for consistency.

In [ ]:
# update to title case
df['Admission Type'] = df['Admission Type'].str.title()
df['Admission Type'] = df['Admission Type'].str.replace('Covid', 'COVID').str.replace('Msr', 'MSR').str.replace('Idoc', 'IDOC').str.replace('Edv', 'EDV')

# check the values
pd.set_option('display.max_rows', 60)
print(df['Admission Type'].value_counts(dropna=False).sort_index())

# update values for consistency
df.loc[df['Admission Type'] == 'Conditional Release, New Sent', 'Admission Type'] = 'Conditional Release, New Sentence'
df.loc[df['Admission Type'] == 'Work Release Violator, New Sentnce', 'Admission Type'] = 'Work Release Violator, New Sentence'

# replace commas with hyphens
df['Admission Type'] = df['Admission Type'].str.replace(',', ' -')

# replace None with nan
df.loc[df['Admission Type'] == 'None', 'Admission Type'] = np.nan




**Admission Type - Changed Labels**

IDOC changed their labels during certain quarters in more recent data. Check which changed and categorize appropriately.


In [ ]:
change_df = df[df['Doc Date'].apply(lambda x: x.date()) >= date(2020, 1, 1)]

# for recent data, plot the number of individuals with each admission type
plot_df = change_df[['Admission Type', 'Doc Date', 'IDOC #']].groupby(['Admission Type', 'Doc Date']).count().reset_index()

# keep only the admission types with significant numbers
plot_df = plot_df[plot_df['IDOC #'] > 10]

# convert dates to strings
plot_df['Doc Date'] = plot_df['Doc Date'].apply(lambda x: dt.strftime(x, '%Y-%m-%d'))

# get doc_dates for x axis
doc_dates = plot_df[['Doc Date']].drop_duplicates()

# plot bar charts for admission types each quarter
fig, axs = plt.subplots(len(plot_df['Admission Type'].unique()), figsize=(8, 50), sharex=True)#, sharey=True)
i = 0

for at in plot_df['Admission Type'].unique():
    # get the relevant df ordered by doc date
    at_df = plot_df[plot_df['Admission Type'] == at]
    at_df = pd.merge(doc_dates, at_df, on='Doc Date', how='left')
    date_order = sorted(doc_dates['Doc Date'].unique())
    at_df = at_df.set_index(at_df['Doc Date']).loc[date_order]
    
    # plot the relevant df
    axs[i].bar(at_df['Doc Date'], at_df['IDOC #'])
    axs[i].title.set_text(at)
    
    i += 1
    
plt.xticks(rotation=90)
plt.show()



For Jun 2021 going forward, IDOC used different labels.
* Before Jun 2021
    * Direct From Court
    * Admit From Other Custody
    * MSR Violator - New Sentence
    * Parole Violator - New Sentence
    * Discharged & Recommitted
    * Conditional Release Violator
    * Technical MSR Violator
    * Technical Parole Violator
    * Transferred from Juvenile
    * Return Additional Mittimus
    * COVID-19 County Admit
* Jun 2021 forward
    * Court Admissions
    * New Sentence Violators
    * Technical Violators
    * Other

In [ ]:
# prep data for a sankey diagram to show the relationship between the labels in the two time periods

sankey_df = df[(df['Doc Date'].apply(lambda x: x.date()) >= date(2020, 3, 1)) & (df['Doc Date'].apply(lambda x: x.date()) <= date(2022, 9, 30))]
sankey_df = sankey_df[sankey_df['Admission Type'] != 'None']

# convert dates to strings
sankey_df['Doc Date'] = sankey_df['Doc Date'].apply(lambda x: dt.strftime(x, '%Y-%m-%d'))

# break into before and after
sankey_df = sankey_df[['IDOC #', 'Current Admission Date', 'Admission Type', 'Doc Date']].drop_duplicates()
after_df = sankey_df[sankey_df['Doc Date'].apply(lambda x: dt.strptime(x, '%Y-%m-%d').date()) >= date(2021, 6, 1)]
before_df = sankey_df[~sankey_df.index.isin(after_df.index)]

# recombine with before and after columns
after_df = after_df.drop('Doc Date', axis=1).rename(columns={'Admission Type': 'After Admission Type'})
before_df = before_df.drop('Doc Date', axis=1).rename(columns={'Admission Type': 'Before Admission Type'})
sankey_df = pd.merge(before_df, after_df, on=['IDOC #', 'Current Admission Date'], how='outer')

# drop dupes and count totals for each before/after combo
sankey_df = sankey_df.drop_duplicates()[['Before Admission Type', 'After Admission Type', 'IDOC #']].groupby(['Before Admission Type', 'After Admission Type']).count().reset_index().rename(columns={'IDOC #': 'Count'})
print(sankey_df)

# get the node df - a df dictionary of admission types and indices
node_df = pd.concat([sankey_df[['Before Admission Type']].rename(columns={'Before Admission Type': 'AT'}),
                       sankey_df[['After Admission Type']].rename(columns={'After Admission Type': 'AT'})])
node_df = node_df.drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index': 'ID'})
node_df['Color'] = get_hex(len(node_df))

# get the links df
links_df = pd.merge(sankey_df, 
                    node_df[['AT', 'ID', 'Color']].rename(columns={'ID': 'Source', 'Color': 'Link Color'}), 
                    left_on='Before Admission Type', right_on='AT', how='inner')
links_df = pd.merge(links_df, 
                    node_df[['AT', 'ID']].rename(columns={'ID': 'Target'}), 
                    left_on='After Admission Type', right_on='AT', how='inner')
links_df['Link Color'] = links_df['Link Color'].apply(lambda x: 'rgba' + str(get_rgb_from_hex(x) + (0.5,)))
links_df = links_df[['Source', 'Target', 'Link Color', 'Count']]



In [ ]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 20,
        thickness = 30,
        line = dict(
            color = 'black',
            width = 0
        ),
        label = node_df['AT'],
        color = node_df['Color']
    ),
    link = dict(
        source = links_df['Source'],
        target = links_df['Target'],
        value = links_df['Count'],
        color = links_df['Link Color']
    )
)])

fig.update_layout(title_text='Change in Admission Type in Jun and Dec 2021',
                  height=1000,
                  font_size=12)
fig.show()

For the most part, the new admission types group multiple old admission types.
* Court Admissions
    * Direct From Court
    * Discharged & Recommitted
    * Return Additional Mittimus
    * COVID-19 County Admit (but let's keep this as its own since its timeframe is so limited)
    * Transferred From Juvenile
* Other
    * Admit from Other Custody
    * Conditional Release Violator
    * Not In IDOC Custody
    * Return To Custody
* New Sentence Violators
    * MSR Violator - New Sentence
    * Parole Violator - New Sentence
* Technical Violators
    * Technical MSR Violator
    * Technical Parole Violator
    
Let's create a new field with the new admission types grouping the old admission types. Group others (from past quarters) based on similarity to other old admission types.


In [ ]:
ct_admis_list = ['Direct From Court', 'Discharged & Recommitted', 'Return Additional Mittimus', 'Transferred From Juvenile']
other_list = ['Admit From Other Custody', 'Conditional Release Violator', 'Not In IDOC Custody', 'Return To Custody', 'Work Release Violator', 'EDV', 'Juv Parole To Adult Parole']
new_sent_viol_list = ['MSR Violator - New Sentence', 'Parole Violator - New Sentence', 'Conditional Release - New Sentence', 'Work Release Violator - New Sentence']
tech_viol_list = ['Technical MSR Violator', 'Technical Parole Violator']

df['New Admission Type'] = None
df.loc[df['Admission Type'].isin(ct_admis_list), 'New Admission Type'] = 'Court Admissions'
df.loc[df['Admission Type'].isin(other_list), 'New Admission Type'] = 'Other'
df.loc[df['Admission Type'].isin(new_sent_viol_list), 'New Admission Type'] = 'New Sentence Violators'
df.loc[df['Admission Type'].isin(tech_viol_list), 'New Admission Type'] = 'Technical Violators'

df.loc[pd.isnull(df['New Admission Type']), 'New Admission Type'] = df.loc[pd.isnull(df['New Admission Type']), 'Admission Type']

print(df['New Admission Type'].value_counts().sort_index())



**Admission Type - Categorization**

Bucket into first-time vs. second-time vs. technical offender. 

NOTE: This is outdated and the logic cannot be used with new IDOC admission types.


In [ ]:
first_list = ['DIRECT FROM COURT', 'TRANSFERRED FROM JUVENILE', 'JUV PAROLE TO ADULT PAROLE']
second_list = ['DISCHARGED & RECOMMITTED', 'MSR VIOLATOR - NEW SENTENCE', 'PAROLE VIOLATOR - NEW SENTENCE', 'WORK RELEASE VIOLATOR - NEW SENTENCE', 'CONDITIONAL RELEASE - NEW SENTENCE']
tech_list = ['TECHNICAL PAROLE VIOLATOR', 'TECHNICAL MSR VIOLATOR', 'CONDITIONAL RELEASE VIOLATOR', 'TECHNICAL VIOLATORS', 'WORK RELEASE VIOLATOR']

df['Admission Type - Recidivism'] = None
df.loc[df['Admission Type'].str.upper().isin(first_list), 'Admission Type - Recidivism'] = 'First IDOC Sentence'
df.loc[df['Admission Type'].str.upper().isin(second_list), 'Admission Type - Recidivism'] = 'Second/+ IDOC Sentence'
df.loc[df['Admission Type'].str.upper().isin(tech_list), 'Admission Type - Recidivism'] = 'Technical MSR/Parole Violator'

df.loc[pd.isnull(df['Admission Type - Recidivism']), 'Admission Type - Recidivism'] = 'Unknown/Uncategorized'

print(df['Admission Type - Recidivism'].value_counts())



**Holding offenses**

There are a lot of similar but slightly different values here. They need to be bucketed/categorized.
* Some can be bucketed based on the JHA interns' list.
* For the rest, bucket based on keywords found in the holding offense.
    * The order of operations matters. E.g. GUN --> weapon offense must come before POSS --> drug offense, since many weapons offenses include the string POSS.
* There is already a Holding Offense Category field (which is very specific), and an Offense Type field (which is broad), but neither are available before Sep 2021.

In [ ]:
print(df['Holding Offense Category'].value_counts(dropna=False))
print(df['Offense Type'].value_counts(dropna=False))

len1 = len(df)

df['Holding Offense'] = df['Holding Offense'].apply(lambda x: str(x).strip())


# specific holding offenses from interns' list
inj_death_list = ['AB/NEGLECT ELD/DEATH/CAREGIVER', 'AGG BATTERY CHILD <13/GREAT BOD HARM', 'AGG BATTERY OF A CHILD', 'AGG BATTERY W/FIREARM/PERSON', 'AGG BATTERY/CHILD <13/PERM DISABL', 'AGG BATTERY/DISCHARGE FIREARM', 'AGG BATTERY/FIREARM/OFF/FIREMAN', 'AGG BATTERY/GREAT BOD HARM/60+', 'AGG BATTERY/GREAT BOD HARM/FLAME SUBST', 'AGG BATTERY/GREAT BOD HARM/PC OFF', 'AGG BATTERY/GREAT BODILY HARM', 'AGG BATTERY/HARM/PEACE OFFICER', 'AGG BATTERY/MERCHANT', 'AGG BTRY/UNLAWFUL DEL/CON SUB', 'AGG KIDNAPING/INFLICT HARM', 'AGG STALKING/BODILY HARM', 'AGG STALKING/BODILY HARM/2ND', 'AGG VEH HIJACKING/DISCH/HARM', 'AGGRAVATED BATTERY W/FIREARM', 'ARMED ROBBERY/DISCHARGE/HARM', 'DOM BTRY/BOD HARM/4+ PRI CONV', 'DOMESTIC BATTERY/BODILY HARM', 'HEINOUS BATTERY', 'HOME INVASION/CAUSE INJURY', 'HOME INVASION/DISCH FIREARM/HARM', 'INTIMIDATION/PHYSICAL HARM', 'MURDER/2ND DEGREE MURDER', 'MURDER/2ND DEGREE/PROVOCATION', 'MURDER/2ND DEGREE/UNREASON', 'MURDER/HOMICIDE/UNBORN CHILD', 'MURDER/INTENT TO KILL/INJURE', 'MURDER/OTHER FORCIBLE FELONY', 'MURDER/STRONG PROB KILL/INJURE', 'RECKLESS CONDUCT/GREAT BODILY HARM', 'VOLUN MANSL/NEGL-DEATH ANOTHER', 'DRUG INDUCED HOMICIDE', 'AGG DUI BODILY HARM/.10+/CHILD', 'AGG DUI/ACCIDENT/BODILY HARM', 'AGG DUI/ACCIDENT/DEATH', 'AGG DUI/BODILY HARM W/CHILD', 'AGG DUI/DEATH OF ANOTHER', 'AGG DUI/GREAT BODILY HARM', 'INVOL MANSLAUGHTER ATV/BOAT', 'INVOL MANSLAUGHTER FAMILY/BOAT', 'RECKLESS HOMIC/INCLINE/DEATH 2+', 'RECKLESS HOMIC/TRAFFIC CONTROL', 'RECKLESS HOMICIDE', 'RECKLESS HOMICIDE/.08 ALC/DRUG', 'RECKLESS HOMICIDE/ALC/CONVIC', 'RECKLESS HOMICIDE/ALC/DRUG', 'RECKLESS HOMICIDE/KILL 2 INDIV', 'SNOWMB OUI OF ALCOHOL/DEATH', 'INVOL MANSLAUGHTER/FAMILY MEMBER', 'INVOLUNTARY MANSLAUGHTER']
inj_death_unclear_list = ['AGG ASLT PEACE OFF/FIRE/ER WRK', 'AGG ASLT/OP MOTOR VEH/STRUCK', 'AGG ASLT/STATE IL EMP/WEAPON', 'AGG ASSAULT/CORRECTIONAL EMP', 'AGG ASSAULT/DHS EMPLOYEE', 'AGG ASSAULT/DISCH FIREARM MV', 'AGG ASSAULT/OP MOTOR VEH/PC OFF', 'AGG ASSAULT/USE FOREARM/PEACE OFF', 'AGG BATTERY SR CITIZEN >60 YRS', 'AGG BATTERY/CONTROLLED SUB', 'AGG BATTERY/GOVERNMENT EMP', 'AGG BATTERY/INGEST TOXIC SUBST', 'AGG BATTERY/JUDGE/EMT', 'AGG BATTERY/PEACE OFF/FIREMAN', 'AGG BATTERY/PEACE OFFICER', 'AGG BATTERY/PREGNANT/HANDICAPPED', 'AGG BATTERY/PUBLIC PLACE', 'AGG BATTERY/STRANGLE/PREV CONV', 'AGG BATTERY/TRANSIT EMPLOYEE', 'AGG BATTERY/USE DEADLY WEAPON', 'AGG BATTERY/WEAPON/NO FIREARM', 'AGG DISCH FIREARM/PC OFF/FIREMAN', 'AGG DISCH FIREARM/VEH/PC OFF/FRMAN', 'AGG DISCH SILENCER PERSON/VEH', 'AGG DISCHARGE FIREARM/BLDG', 'AGG DISCHARGE FIREARM/OCC BLDG', 'AGG DISCHARGE FIREARM/OCC VEH', 'AGG DOMESTIC BATTERY/STRANGLE', 'AGG INTIMIDATION PEACE OFFICER', 'AGG KIDNAPING ARMED W FIREARM', 'AGG KIDNAPING/CHILD<13/RETARDED', 'AGG KIDNAPING/CONCEAL IDENT', 'AGG ROBBERY/CONTROLLED SUB', 'AGG ROBBERY/INDICATE ARM W/FIR', 'AGG STALKING/VIO REST ORDER', 'AGG VEHICULAR HIJACKING/DISCHARGE', 'AGG VEHICULAR HIJACKING/FIREARM', 'AGG VEHICULAR HIJACKING/HANDICAPPED', 'AGG VEHICULAR HIJACKING/PASS <16 YRS', 'AGG VEHICULAR HIJACKING/WEAPON', 'AGGRAVATED ASSAULT', 'AGGRAVATED BATTERY/NURSE', 'AGGRAVATED BATTERY/STRANGLE', 'AGGRAVATED BATTERY/VICTIM 60+', 'AGGRAVATED DISCHARGE/FIREARM', 'AGGRAVATED DOMESTIC BATTERY', 'AGGRAVATED KIDNAPING/ARMED', 'AGGRAVATED KIDNAPING/RANSOM', 'AGGRAVATED ROBBERY', 'AGGRAVATED TRAFFICKING/BENEFIT', 'AGGRAVATED UNLAWFUL RESTRAINT', 'AGGRAVATED VEHICULAR HIJACKING', 'ARMED ROBBERY', 'ARMED ROBBERY/ARMED W/FIREARM', 'ARMED ROBBERY/DISCH FIREARM', 'ARMED ROBBERY/NO FIREARM', 'ARMED VIOL/CATEGORY I WEAPON', 'ARMED VIOL/CATEGORY II WEAP/1ST', 'ARMED VIOL/CATEGORY II WEAPON', 'ARMED VIOL/CATEGORY III WEAPON', 'CAUSE CHILD/ENDANGERED/2+', 'CHILD ABDUCTN/CONCEAL/DETAIN', 'COMMUNICATE/DETAIN WITNESS', 'CRIM TRES TO RES/PERS PRESENT', 'DISMEMBERING A HUMAN BODY', 'DOM BTRY/CONTACT/3 PRIOR CONV', 'DOMESTIC BATTERY/ CONTACT/PRIOR', 'DOMESTIC BATTERY/PHY CONTACT/2ND+', 'DOMESTIC BATTERY/VIOL PROTECT ORDER', 'GIVE FALSE BOMB/GAS ALARM', 'HABITUAL CRIMINAL/2 PRIOR CL X', 'HARASS JURORS/WITNESSES', 'HARASS WITNESS', 'HARASSMENT BY PHONE/4TH +', 'HOME INVASION', 'HOME INVASION/ARMED W/FIREARM', 'HOME INVASION/ARMED/FORCE', 'HOME INVASION/DANDEROUS WEAP', 'HOME INVASION/DISCH FIREARM', 'HOME INVASION/FIREARM', 'HOME INVASION/SEX OFFENSE', 'HOME INVASION/VIO AGAINST PERS', 'HRSMT/PREVENT/SERV/SAME VIC', 'HRSMT/THREATEN PERSON/KILL', 'HUMAN TRAFFICKING FOR LABOR/BENEFITS', 'INTIMIDATION/CONTEMPT/RIDICULE', 'INTIMIDATION/CRIMINAL OFFENSE', 'INVOL SERVITUDE/THREAT PHYS HARM', 'KIDNAPING ARMED WITH FIREARM', 'KIDNAPING DISCHARGE FIREARM', 'KIDNAPING W FORCE OR THREAT', 'KIDNAPING/SECRETLY CONFINE', 'PHONE HRSMT/LEWD/THREAT KILL', 'RECKLESS DISCH FIREARM/ENDANGERS', 'ROBBERY', 'ROBBERY/SCHOOL/PLACE WORSHIP', 'ROBBERY/VIC HANDICAP / 60+', 'ROBBERY/VIC HANDICAP OR 60+ YR', 'STALKING BY TRANSMITTING THREAT', 'STALKING/PERSON/SURVEILLANCE', 'STALKING/TRANSMITS THREAT/2ND', 'UNLAWFUL RESTRAINT', 'UNLAWFUL VEHICULAR INVASION', 'VEHICULAR HIJACKING', 'VIO ORDER/PRIOR VIO OF ORDER', 'VIOLATE ORDER/PRIOR DOM BATTERY', 'AB/NEGLECT ELDERLY/CAREGIVER', 'ARMED VIOL/CATEGORY II WEAP/2ND+', 'CONTEMPT', 'KIDNAPING/DECEIT OR ENTICE', 'VIOLATE STALKING NO CONTACT/2+', 'CHILD ABDUCT/LURE/VIC<17/PRIOR', 'WINDOW PEEPING 3RD+', 'ATT AGG BATTERY/CHILD <13/PERM DISABL', 'ATT AGG BATTERY/PEACE OFFICER', 'ATT AGG DISCHARGE FIREARM', 'ATT AGG DISCHARGE FIREARM AT VEH', 'ATT AGG DISCHARGE FIREARM OFFICER', 'ATT AGG VEHICLE HIJACK W WEAPON', 'ATT ARMED ROBBERY/ARMED', 'ATT ARMED ROBBERY/NO FIREARM', 'ATT DRUG-INDUCED HOMICIDE', 'ATT HOME INVASION/ARMED W/FIREARM', 'ATT HOME INVASION/FIREARM', 'ATTEMPT AGG BATTERY OF A CHILD', 'ATTEMPT AGG BATTERY/DISCHARGE FIREARM', 'ATTEMPT AGG DISCHARGE/FIREARM', 'ATTEMPT AGG KIDNAPG/CHILD<13/RETARD', 'ATTEMPT AGG KIDNAPING/INFLICT HARM', 'ATTEMPT AGG ROBBERY/INDICATE ARM W/FIR', 'ATTEMPT AGG VEHICULAR HIJACKING', 'ATTEMPT AGGRAVATED ROBBERY', 'ATTEMPT ARMED ROBBERY', 'ATTEMPT ARMED VIO/CATEGORY I WEAPON', 'ATTEMPT DISARM PC OFF/CORR EMP', 'ATTEMPT HARASS JURORS/WITNESSES', 'ATTEMPT HOME INVASION/CAUSE INJURY', 'ATTEMPT INTENT/HOMICIDE/UNBORN CHILD', 'ATTEMPT KIDNAP ARMED W FIREARM', 'ATTEMPT MURDER/INTENT TO KILL/INJURE', 'ATTEMPT MURDER/OTHER FORCIBLE FELONY', 'ATTEMPT MURDER/STRONG PROB KILL/INJURE', 'ATTEMPT ROBBERY', 'ATTEMPT ROBBERY/VIC HANDICAP OR 60+ YR', 'ATTEMPT SOLICITATION/MURDER/FOR HIRE', 'ATTEMPT VEHICULAR HIJACKING', 'CONSP AGGRAVATED ROBBERY', 'CONSP ARMED ROBBERY', 'CONSP MURDER/INTENT TO KILL/INJURE', 'SOLICIT MURDER/INTENT TO KILL/INJURE', 'SOLICITATION ROBBERY', 'SOLICITATION/MURDER', 'SOLICITATION/MURDER/FOR HIRE', 'CONSP ROBBERY', 'ATTEMPT AGG KIDNAP/CONCEAL IDENTITY', 'ATTEMPT KIDNAPING/SECRETLY CONFINE', 'CONSP AGG BATTERY/DISCHARGE FIREARM']
sex_list = ['AGG CHILD PORN/DISSEM FILM', 'AGG CHILD PORN/EXHIBIT', 'AGG CHILD PORN/LEWD EXHIBITION', 'AGG CHILD PORN/PERSON/ANIMAL', 'AGG CHILD PORN/POSS FILM/PHOTO', 'AGG CHILD PORN/POSS FILM/PREV', 'AGG CRIM SEX AB/VIC <18/FAMILY', 'AGG CRIM SEX ABUSE', 'AGG CRIM SEX ABUSE/<5 YR VIC', 'AGG CRIM SEX ABUSE/BODILY HARM', 'AGG CRIM SEX ABUSE/DURING FELONY', 'AGG CRIM SEX ABUSE/FAMILY', 'AGG CRIM SEX ABUSE/FORCE/VIC 9-13', 'AGG CRIM SEX ABUSE/INTELLIG DISABL', 'AGG CRIM SEX ABUSE/VIC <13', 'AGG CRIM SEX ABUSE/VIC 13-16', 'AGG CRIM SEX ABUSE/VIC 13-17', 'AGG CRIM SEX ABUSE/VICTIM <13', 'AGG CRIM SEX ABUSE/VICTIM <9', 'AGG CRIM SEX ASLT/INTEL DISABL', 'AGG CRIM SEX ASLT/THREAT LIFE', 'AGG CRIM SEX ASSAULT/BODILY HARM', 'AGG CRIM SEX ASSAULT/FELONY', 'AGG CRIM SEX ASSAULT/FIREARM', 'AGG CRIM SEX ASSAULT/FORCE VIC9-13', 'AGG CRIM SEX ASSAULT/HANDICAPPED', 'AGG CRIM SEX ASSAULT/RETARDED', 'AGG CRIM SEX ASSAULT/THREAT LIFE', 'AGG CRIM SEX ASSAULT/VIC 60+', 'AGG CRIM SEX ASSAULT/VICTIM <13', 'AGG CRIM SEX ASSAULT/VICTIM <9', 'AGG CRIM SEX ASSAULT/VICTIM >60', 'AGG CRIM SEX ASSAULT/WEAPON', 'AGG CRIM SEXUAL ABUSE/FELONY', 'AGG CRIM SEXUAL ASSAULT/CON SUB', 'AGG CRIM SX AB/VIC 13<18/TRUST', 'AGG CRIM SX ASLT/FORCE VIC9-13', 'AGG IND LIB/CHILD/INFLICT HARM', 'CHILD PORN/FILM/TAPE/PHOTO/ACT', 'CHILD PORN/MOVING DEPICTION', 'CHILD PORN/PERS/ANIM/MOV DPTN', 'CHILD PORN/PERSON/ANIMAL', 'CHILD PORN/POSE/EXHIBITION', 'CHILD PORN/POSS FILM/PHOTOS', 'CHILD PORN/POSS PHOTO/VIC <13', 'CHILD PORN/POSS/MOVING DPTN', 'CHILD PORN/REPRODUCE/MOV DPTN', 'CHILD PORN/REPRODUCE/SELL', 'CHILD PORN/SOL CHILD/MOV DPTN', 'CHILD PORN/SOL/CHILD/APPEAR', 'CHILD PORNOGRAPHY/VICTIM <13', 'CRIM SEX ABUSE/CANT CONSENT/2ND', 'CRIM SEX ASLT/FAM MBR <18/2+', 'CRIM SEX ASLT/FAMILY MEMBER<18', 'CRIM SEX ASLT/FORCE/PREV CONV', 'CRIM SEX ASSAULT/CANT CONSENT', 'CRIM SEX ASSAULT/CANT CONSENT/2ND', 'CRIM SEX ASSAULT/FAMILIES', 'CRIM SEX ASSAULT/FAMILIES/2ND+', 'CRIM SEX ASSAULT/FORCE', 'CRIM SEX ASSAULT/FORCE/2ND+', 'CRIM SEX ASSAULT/SUPERVN VIC 13-17', 'CRIM SEX ASSAULT/VICTIM 13-17', 'CRIM SEX ASSAULT/VICTIM 13-17/2ND', 'CRIM SEXUAL ABUSE/CONSENT', 'CRIM SEXUAL ABUSE/CONSENT/2+', 'CRIMINAL SEX ASSAULT/CONSENT', 'CRIMINAL SEXUAL ABUSE/FORCE', 'DEVIATE SEXUAL ASSAULT', 'GROOMING', 'INDECENT SOL ADULT/<13/PENETRATION', 'INDECENT SOL/AGG CRIM SEX ABUSE', 'INDECENT SOL/CRIM SEX ASSAULT', 'INDECENT SOL/PREDITORY/AGG SEX', 'NONCONSENSUAL  DISSM SEX IMAGE', 'PERMIT SEXUAL ABUSE OF CHILD', 'PRED CRIM SEX ASSLAUT/VICTIM <13', 'PREDATORY CRIM SEX ASSAULT/FIREARM', 'PREDATORY CRIM SEX ASSAULT/HARM', 'PREDATORY CRIMINAL SEXUAL ASSAULT', 'PROM JUV PROST/MINOR <13/RETAR', 'PUBLIC INDECENCY/EXPOSURE/3+', 'RAPE', 'RAPE PRIOR TO 2/1/78', 'SEX RELATIONS WITHIN FAMILIES', 'SEXUALLY DANGEROUS PERSON', 'TRAFFIC SEX SERV MINOR < 17', 'TRAVELING TO MEET A MINOR', 'UNLAWFUL VIDEO/VIC<18/SEX OFF', 'EXPLOIT CHILD/SEX ACT/2ND', 'CUSTODIAL SEXUAL MISCONDUCT', 'INDECENT SOLICIT CHILD/INTERNET', 'ATT AGG CRIM SEX ASSAULT VIC 60+', 'ATT CRIM SEX ASSAULT/FORCE', 'ATT PERMIT SEXUAL ABUSE OF CHILD', 'ATTEMPT AGG CRIM SEX ASLT/BODILY HARM', 'ATTEMPT AGG CRIM SEX ASSAULT', 'ATTEMPT AGG CRIM SEX ASSAULT/BODILY HARM', 'ATTEMPT AGG CRIM SEX ASSAULT/FELONY', 'ATTEMPT AGG CRIM SEX ASSAULT/VICTIM >60', 'ATTEMPT AGG CRIM SEX ASSAULT/WEAPON', 'ATTEMPT AGG CRIM SX AB/>5 YR OLDER VIC', 'ATTEMPT CRIM SEX ASSAULT/FORCE', 'ATTEMPT CRIM SEX ASSAULT/VICTIM 13-17', 'ATTEMPT PRED CRIM SEX ASLT/VICTIM <13', 'ATTEMPT PRED CRIMINAL SEXUAL ASSAULT', 'SOL AGG CRIM SEX ASSAULT/VIC 60+']
drugs_list = ['15<200 OBJECT/PARTS LSD/ANALOG', 'AGG DEL METH PROTECTED <5 GRAMS', 'AGG DEL METH/PROTECTED/100+ GR', 'AGG DEL METH/PROTECTED/5<15 GRAMS', 'AGG METH MANU/ORGANIZE/100<400 GR', 'AGG METH MANU/PROTECTED<15', 'AGG METH MANU/WORSHIP/<15 GR', 'AGG METH MANUF/APT/<15 GRAMS', 'AGG METH MANUF/APT/100<400 GR', 'AGG METH MANUF/CHILD 15<100 GRAMS', 'AGG METH MANUF/CHILD/<15 GRAMS', 'AGG METH MANUF/CHLD/100<400 GRAMS', 'AGG METH MANUF/CHLD/400+ GR', 'AGG METH MANUF/ORGANIZE/<15 GR', 'CALCULATED CRIM DRUG CONSP', 'CALCULATED CRIM DRUG CONSPIR', 'CANNABIS TRAFFICKING', 'CONT SUBS ACT-MANU/DEL', 'CONTROLLED SUB TRAFFICKING', 'CRIM DRUG CONSPIRACY', 'CRIMINAL DRUG CONSPIRACY', 'CRIMINAL DRUG CONSPIRACY >100 GR', 'DEL CONT SUB<18/PARK/SCHOOL/PUB HOUS', 'DEL CONT/COUNT SUB TO <18', 'DEL METH <5 GRAMS', 'DEL/NON NARC SCHED I/II/SC/HS/PARK', 'DISPOSE METH MANUF WASTE', 'MAN/DEL OTHER AMOUNT AMPHETAMINE', 'MANU 5>15 GRAMS ECSTASY', 'MANU/DEL 01-15 GRAMS COCAINE', 'MANU/DEL 1-14 GRAMS HEROIN', 'MANU/DEL 1<15 GRAM FENTANYL', 'MANU/DEL 10-15 GRAMS HEROIN', 'MANU/DEL 10<15 OBJECTS/PARTS LSD', 'MANU/DEL 10>15 PILLS ECSTASY', 'MANU/DEL 100<400 GR FENTANYL', 'MANU/DEL 100<400 GRAMS COCAINE', 'MANU/DEL 100<400 GRAMS HEROIN', 'MANU/DEL 100<400 GRAMS LSD', 'MANU/DEL 100>400 GR ECSTASY', 'MANU/DEL 15/+ GRAMS HEROIN', 'MANU/DEL 15/+GRAMS COCAINE', 'MANU/DEL 15<100 GRAMS COCAINE', 'MANU/DEL 15<100 GRAMS HEROIN', 'MANU/DEL 15<100 GRAMS LSD', 'MANU/DEL 15>100 GR ECSTASY', 'MANU/DEL 15>200 PILLS ECSTASY', 'MANU/DEL 1500+ PILLS ECSTASY', 'MANU/DEL 200+ SCHED I&II', 'MANU/DEL 200+GRAMS METH/AMPH', 'MANU/DEL 200>600 PILLS ECSTASY', 'MANU/DEL 2ND OR SUBQ OFFENSE IN ACT', 'MANU/DEL 3-15 GRAMS LSD', 'MANU/DEL 400<900 GRAMS COCAINE', 'MANU/DEL 400<900 GRAMS METH', 'MANU/DEL 50-200 GRAMS SCHED I&II', 'MANU/DEL 600<1500 OBJECTS/PARTS LSD', 'MANU/DEL 900+ GRAMS COCAINE', 'MANU/DEL 900+ GRAMS ECSTASY', 'MANU/DEL 900+ GRAMS HEROIN', 'MANU/DEL AMT NARC SCHED I/II/SCHOOL/HS/PARK', 'MANU/DEL BARB ACID/SC/PUB HOUS/PARK', 'MANU/DEL CANNABIS/>5,000 GRAMS', 'MANU/DEL CANNABIS/>500 GRAMS', 'MANU/DEL CANNABIS/10-30 GRAMS', 'MANU/DEL CANNABIS/2000<5000 GRAMS', 'MANU/DEL CANNABIS/30-500 GRAMS', 'MANU/DEL COCAINE/SCHOOL/PUB HOUS/PARK', 'MANU/DEL CONT SUBS (PRIOR 1/1/90)', 'MANU/DEL CONT SUBS/ENHANCED', 'MANU/DEL HEROIN/SCHOOL/PUB HOUS/PARK', 'MANU/DEL OTHER AMOUNT SCHEDULE III', 'MANU/DEL OTHER AMOUNT SCHEDULE IV', 'MANU/DEL OTHER AMT METH', 'MANU/DEL OTHER AMT NARC SCHED I&II', 'MANU/DEL OTHER AMT NARCOTIC SCHED I&II', 'MANU/DEL OTHER NON-NARCOTIC SCHED I&II', 'MANU/DEL SCHED I/II/SCH/HS/PARK', 'MANU/DISTRIB LOOK-ALIKE SUB', 'METH CONSP CONT SUBS >100 GR', 'METH DELIVERY/100<400 GR', 'METH DELIVERY/15<100 GRAMS', 'METH DELIVERY/400<900 GR', 'METH DELIVERY/5<15 GRAMS', 'METH DELIVERY/900+ GR', 'METH DELIVERY<5 GRAMS', 'METH MANUFACTURE/100<400 GR', 'METH MANUFACTURE<15 GRAMS', 'METH MANUFACTURING/100<400 GRAMS', 'METH MANUFACTURING/15<100 GR', 'METH MANUFACTURING/15<100 GRAMS', 'METH MANUFACTURING/400<900 GR', 'METH MANUFACTURING/400<900 GRAMS', 'METH MANUFACTURING/900+ GR', 'METH MANUFACTURING<15 GRAMS', 'METH PRECURSOR TRAFFICKING', 'METH PRECURSOR/15<30 GRAMS', 'METH PRECURSOR/500+ GRAMS', 'METH PRECURSOR<15 GRAMS', 'METHAMPHETAMINE CONSPIRACY', 'METHAMPHETAMINE TRAFFICKING', 'MFG/DEL 100<400 GR HERO/ANLG', 'MFG/DEL 15<100 GR FENTANYL', 'MFG/DEL 15<100 GR HEROIN/ANLG', 'MFG/DEL 400<900 GR HERO/ANLG', 'NARCOTICS RACKETEERING', 'OBTAIN SUBSTANCE BY FRAUD/1ST', 'POSS AMT CON SUB EXCEPT(A)/(D)', 'POSS CANNABIS/>5,000 GRAMS', 'POSS CANNABIS/10-30 GRAM/SUBQ', 'POSS CANNABIS/2,000<5,000 GRAMS', 'POSS CANNABIS/500<2,000 GRAMS', 'POSS CONT SUBS', 'POSS HYPO/SYRINGE/NEEDLES/2ND+', 'POSS METH MANUF MATERIAL', 'POSS OF METH < 5 GRAMS', 'POSS OF METH/ 15<100 GRAMS', 'POSSESS 100<400 GRAMS COCAINE', 'POSSESS 100<400 GRAMS HEROIN', 'POSSESS 15+ GRAMS COCAINE', 'POSSESS 15+ GRAMS HEROIN', 'POSSESS 15+ GRAMS MORPHINE', 'POSSESS 15<100 GRAMS COCAINE', 'POSSESS 15<100 GRAMS HEROIN', 'POSSESS 15<100 GRAMS METH', 'POSSESS 15<200 OBJECT/PART LSD', 'POSSESS 15>200 PILLS ECSTASY', 'POSSESS 1500+ PILLS ECSTASY/ANALOG', 'POSSESS 200+ GRAMS OTHER SCHED I&II', 'POSSESS 400<900 GRAMS COCAINE', 'POSSESS 400<900 GRAMS HEROIN', 'POSSESS 900 + GRAMS COCAINE', 'POSSESS 900 + GRAMS HEROIN', 'POSSESSION OF METH/100<400 GRAMS', 'POSSESSION OF METH/15<100 GRAMS', 'POSSESSION OF METH/400<900 GR', 'POSSESSION OF METH/5<15 GRA', 'POSSESSION OF METH/900+ GR', 'POSSESSION OF METH< 5 GRAMS', 'PRODUCE >200 CANNABIS PLANTS', 'TAMPER W/ANHYD AMM EQUIPMENT', 'USE OF DANGEROUS PLACE/CON SUB', 'USE VEH/STRUCTURE/PROP/METH', 'POSS BLANK COUNTERFEIT SCRIPT', 'POSS CANNABIS/30-500 GRAM/1ST', 'MANU/DEL CONT SUBS', 'MANU/DEL 10<30 GRAMS KETAMINE', 'POSS 400>900 GR ECSTASY/ANALOG', 'PRODUCE 5-20 CANNABIS PLANTS', 'ATT MANU 200>600 PILLS ECSTASY', 'ATT METH PRECURSOR/30<150 GR', 'ATTEMPT MANU/DEL 15/+GRAMS COCAINE', 'ATTEMPT MANU/DEL CANNABIS/>500 GRAMS', 'ATTEMPT MANU/DEL OTHER AMT NARC', 'ATTEMPT POSS METH MANUF MATERIAL', 'ATTEMPT POSSESS 15<100 GRAMS COCAINE', 'CONSP MANU/DEL 01-15 GRAMS COCAINE', 'CONSP MANU/DEL OTHER AMOUNT METH', 'CONSP MFG/DEL CANNABIS/>5,000 GRAMS', 'SOLICIT MANU/DEL 01-15 GRAMS COCAINE', 'ATTEMPT MANU/DEL 01-15 GRAMS COCAINE', 'CONSPIRACY METH MANUFACTURE<15 GRAMS', 'ATTEMPT POSS AMT CON SUB', 'CONSP MANU/DEL 15/+GRAMS COCAINE']
property_list = ['AGG ARSON', 'AGG ARSON/INJURE FIRE/POLICE', 'AGG ARSON/KNOW PEOPLE PRESENT', 'AGG HOME FRAUD VIC >60 >%500', 'AGG ID THEFT/<$300/DISABLED', 'AGG ID THEFT/$10,000-$100,000/60 YRS+', 'AGG ID THEFT/$300-10,000/60 YRS +', 'AGG ID THEFT/AGE 60+/2+', 'AGG ID THFT/10K-100K/GANG ACTS', 'AGG ID THFT/ID STOLEN/VIC 60+', 'AGG INSURANCE FRAUD', 'AGGRAVATED ARSON', 'AGGRAVATED ARSON/BODILY HARM', 'AID/ABET/POSS/SELL STOLEN VEH', 'ARSON', 'ARSON/REAL/PERSONAL PROP>$1', 'BAD CHK/OBTAIN CON PROP/>150', 'BURGLARY', 'BURGLARY W/O CAUSING DAMAGE', 'BURGLARY/SCHOOL/PLACE WORSHIP', 'CAUSING A CATASTROPHE', 'CONTINUING FINANCE CRIME ENTRPRS', 'COUNTERFEIT CREDIT/DEBIT CARD', 'CRIM DMG TO PROP $300-10000', 'CRIMINAL DAMAGE $10000-$100000', 'DEFRAUD FINANC INST/$500-$10,000', 'DEFRAUD FINANCIAL INST/>$100K', 'FIN EXPLOIT ELDERLY/70+/$15K+', 'FINANCE EXPLOIT ELD/DISABL/$300-$5,000', 'FINANCIAL INSTITUTION ROBBERY', 'FORGERY /MAKE OR ALTER DOCUMENT', 'FORGERY/ISSUE DOCUMENT/1 UPC', 'FORGERY/ISSUE/DELIVER DOCUMENT', 'FORGERY/POSSESS W INTENT', 'ID THEFT/OBTAIN/3+ INDIV', 'ID THFT/OBTAIN INFO/COMMIT FEL', 'IDENTITY THEFT >$100,000', 'IDENTITY THEFT/<$300', 'IDENTITY THEFT/$10K - $100K', 'IDENTITY THEFT/$2000-$10,000', 'IDENTITY THEFT/$300-$2000', 'IDENTITY THEFT/KNOWS ID STOLEN', 'KNOW ID STOLEN/MILITARY VIC/2+', 'MOB ACTION', 'ORGANIZE/AGG VEH THEFT CONSPIR', 'ORGANIZER/FIN CRIME ENTRPRS', 'POSS STOLEN VEHICLE > $25,000', 'POSSESS BURGLARY TOOLS', 'RECEIVE/POSS/SELL STOLEN VEH', 'RECEIVE/POSS/SELL STOLEN VEHICLE', 'RESIDENTIAL ARSON', 'RESIDENTIAL BURGLARY', 'RET THEFT/DISP MERCH/<$300/PREV CONV', 'RET THEFT/DISP MERCH/>$300', 'RETAIL THEFT/DISP MERCH/<$150/2+', 'RETAIL THEFT/EMERGENCY EXIT/>$300', 'RETAIL THEFT/MOTOR FUEL/>$150', 'THEFT', 'THEFT <$300', 'THEFT <$300 PRIOR', 'THEFT <$300 WITH PRIOR', 'THEFT <$300/PRIOR CONVICT', 'THEFT <$300/SCHOOL/WORSHIP', 'THEFT >$10,000 <$100,000', 'THEFT >$10,000-$100,000', 'THEFT >$100,000-$500,000', 'THEFT >$100,000/SCHOOL/WORSHIP', 'THEFT >$300-$10,000', 'THEFT $300-$10,000/SCHOOL/WORSHIP', 'THEFT $300<$10,000', 'THEFT CONTROL INTENT <$500 PRIOR', 'THEFT CONTROL INTENT $500<$10,000', 'THEFT CONTROL INTENT 10K<100K', 'THEFT CONTROL OF PROPERTY <$500', 'THEFT DECEP INTENT $10,000<$100,000', 'THEFT DECEPTION INTENT 300<10,000', 'THEFT DECEPTION INTENT 500<10000', 'THEFT STOLEN <$300 PRIOR', 'THEFT STOLEN INTENT PERS <$300', 'THEFT STOLEN/>$300 <$10,000', 'THEFT/COIN OP MACHINE/2ND+', 'THEFT/CONTROL INTENT>$500K-1M', 'THEFT/CONTROL/INTENT >$1M', 'THEFT/DECEPTION > $100,000', 'THEFT/DECEPTION >$500,000', 'THEFT/DECEPTION/<$500 PRIOR', 'THEFT/DECEPTION/>$500 <10K', 'THEFT/EMER EXIT/<$300/PRECONV', 'THEFT/STOLEN/ <$500 PRIOR CONV', 'THEFT/STOLEN/>$10,000 <$100,000', 'THEFT/UNAUTHD CONTROL>$300<10000', 'THFT/SWITCH PRICE/<300/PRECONV', 'UNLAWFUL POSS CREDIT/DEBIT CARD', 'UNLAWFUL POSS/DRIVER/VEH/STOLEN', 'UNLAWFUL POSS>3VEH/PARTS/STOLEN', 'USE FORGED CREDIT/DEBIT CARD/>$300', 'VEHICLE THEFT CONSPIRACY', 'FORGE REGISTRATION, ETC', 'ID THEFT/USE ID/COMMIT FELONY', 'IDENTITY THEFT $2,000<$10,000', 'THEFT/FALSE REP/<$300/PRECONV', 'USE OF ACCT NUMBER/CODE/>$150', 'AID/ABET CONCEAL/MISREP VEHICLE', 'RETAIL THEFT/EMERGENCY EXIT/<$300', 'HOME REPAIR FRAUD/CONTRACT >$1,000', 'THFT/LES RTN PROP/<300/PRECONV', 'USE OF ACCT NUMBER/CODE/<$150', 'CRIM DAMAGE PROP>$100,000', 'RET THEFT/MOTOR FUEL/<$150/PREV CONV', 'UNLAWFUL USE ID CARD/THEFT', 'ATT RECEIVE/POSS/SELL STOLEN VEH', 'ATTEMPT AGG ARSON', 'ATTEMPT ARSON', 'ATTEMPT ARSON >$150', 'ATTEMPT BURGLARY', 'ATTEMPT BURGLARY/SCHOOL/PLACE WORSHIP', 'ATTEMPT POSS STOLEN TITLE/CERTIF/PLATE', 'ATTEMPT POSS STOLEN VEHICLE > $25,000', 'ATTEMPT RESIDENTIAL ARSON', 'ATTEMPT RESIDENTIAL BURGLARY', 'ATTEMPT THEFT >$10,000-$100,000', 'CONSP RESIDENTIAL BURGLARY', 'ATTEMPT AID/ABET CONCEAL/MISREP VEHICLE']
weapon_list = ['AGG POSS/11-20 STOLEN FIREARMS', 'AGG POSS/2-5 STOLEN FIREARMS', 'AGG UNLAWFUL USE OF WEAPON/VEH', 'AGG UNLAWFUL USE WEAPON/VEH/2ND', 'AGG UUW/ON PERSON', 'AGG UUW/PERS/FIR LOADED/FOID', 'AGG UUW/PERSON/PREV CONVICTION', 'AGG UUW/VEH/PREV CONVICTION', 'ARMED HABITUAL CRIMINAL', 'FIREARM/AMMO ACT-POSS WO/ID CARD', 'FIREARM/FOID INVALID/NO ELIG', 'GUNRUNNING', 'POSS EXPLOSIVE/INCENDIARY DEVICE', 'POSS FIREARM W/ DEFACED SER NO', 'POSS FIREARM/FOID NO ISSUE/NO ELIG', 'POSS FIREARM/LAND/GANG MEMBER', 'POSS OF FIREARM BY GANG MEMBER', 'POSSESSION OF STOLEN FIREARM', 'UNLAWFUL POSS HANDGUN/DEL/<21', 'UUW-FELON POSS WEAPON/BODY ARMOR', 'UUW-FELON POSS/USE FIREARM PRIOR', 'UUW-FELON POSS/USE FIREARM/PAROLE', 'UUW-FELON POSS/USE WEAPON/FIREARM', 'UUW/CARRY/POSS FIREARM/SCHOOL', 'UUW/MACHINE GUN/AUTO WEAPON/VEH', 'UUW/RIFLE <16 IN/SHOTGUN <18', 'UUW/CARRY/POSS FIREARM/2ND+', 'AGG UUW/PERSON/LOADED FIREARM/2+', 'UUW/CARRY/POSS FIREARM/1ST', 'ATT AGG POSS/31+ STOLEN FIREARMS', 'ATT FELON POSS/USE WEAP/BODY ARMOR', 'ATTEMPT AGG UUW W/ BODY ARMOR/PRIVATE', 'ATTEMPT UUW/FELON POSS/USE FIREARM/PAROLE', 'ATT POSS FIREARM/VEHICLE/GANG MEMBER', 'ATTEMPT POSSESSION OF STOLEN FIREARM']
vehicle_list = ['AGG DUI', 'AGG DUI - SCHOOL BUS DRIVER', 'AGG DUI / 5TH DUI', 'AGG DUI / NO VALID INSURANCE', 'AGG DUI LIC SUSP OR REVOKED', 'AGG DUI/ 4TH /BAC 0.16+', 'AGG DUI/3', 'AGG DUI/3/BAC .16+', 'AGG DUI/3/PASS <16', 'AGG DUI/3RD/LIC SUSP OR REV', 'AGG DUI/3RD+', 'AGG DUI/4', 'AGG DUI/4TH/LIC SUSP OR REVOKED', 'AGG DUI/5/BAC 0.16>', 'AGG DUI/6+', 'AGG DUI/6+/BAC 0.16+', 'AGG DUI/DO DRIVER LICENSE', 'AGG DUI/LIC SUSP OR REVOKED', 'AGG DUI/NO VALID DL', 'AGG DUI/NO VALID INS', 'AGGRAVATED DUI/3RD+ DUI', 'DUI/3RD/BAC 0.16+', 'DUI/4TH+/LIC SUSP OR REVOKE', 'DUI/6TH', 'AGG DUI NO VALID DRIVERS LICENSE', 'AGG DUI/3RD+ DUI', 'DRIV LIC REVOKED/RECK HOMIC/2', 'DRIVE REVOKED/RECK HOMIC', 'DRIVE REVOKED/RECKL HOMIC/3', 'DRIVE W REVOKED/RECK HOMIC/4+', 'DRIVING RVK/SUSP DUI/SSS 4-9', 'DRIVING W/ REVOKE/SUSPEND LICENSE 2ND+', 'DRIVING W/ SUSPEND/REVOKE LICENSE 10-14', 'DRIVING W/ SUSPEND/REVOKE LICENSE 15+', 'DRIVING W/ SUSPEND/REVOKE LICENSE 2ND', 'DRIVING W/ SUSPEND/REVOKE LICENSE 3RD', 'DRIVING W/ SUSPEND/REVOKE LICENSE 4-9', 'REVOKED/SUSPENDED 2ND DUI']
registry_list = ['VIOLENT OFF/YOUTH FAIL REGIS NEW RESID/SCHOOL', 'VIOLENT OFF/YOUTH FAIL REGIS/FAIL TO REPORT', 'VIOLENT OFF/YOUTH FAIL REGISTER IN ILLINOIS', 'VIOLENT OFF/YOUTH FAIL REGIS IN ILLINOIS 2+', 'CHIL SEX OFFEN/RESIDE SAY CARE', 'CHILD SEX OFFENDER/PUBLIC PARK', 'CHILD SEX OFFENDER/RESIDE 500FT', 'FAIL REGIS/NEW REDIS/SCH/2ND+', 'FAIL TO REPORT ANNUALLY/2+', 'SEX OFF FAIL REGISTER NEW RESIDENCE', 'SEX OFF FAIL REPORT CHANGE ADDR/EMPL', 'SEX OFF FAIL REPORT CHANGE ADDRESS/EMPLOY', 'SEX OFF FAIL TO REPORT ANNUALLY', 'SEX OFF FAIL TO REPORT WEEKLY/2ND', 'SEX OFFENDER AT PUBLIC PARK', 'SEX OFFENDER GIVE FALSE INFO/2ND', 'SEX OFFENDER REGIS VIOLATION', 'SEX OFFENDER REGIS/FALSE INFO', 'SEX VIOL/DANGEROUS FAIL TO RPT', 'PHOTO/DIGITAL IMAGE OF CHILD BY SEX OFFENDER', 'ATT SEX OFF FAIL REPORT/NO ADDR/2', 'ATT VIOL SEX OFFENDER REGIS ACT']
other_list = ['AGG FLEEING POLICE/21 MPH OVER', 'AGG FLEEING POLICE/2ND', 'AGG FLEEING/2+ CON DVC/2ND', 'AGG FLEEING/2+ DISOBEY TRAFFIC DEVICES', 'AGG FLEEING/21 MPH OVER/2ND', 'AGG FLEEING/DAMAGE >$300 PROPERTY', 'BRIBERY/OFFER BRIBE', 'BRING CANNABIS IN A PENAL INST', 'BRING CON SUBSTANCE PENAL INST', 'BRING TOOL IN A PENAL INSTITUTION', 'BRING/POSS CONTRAB IN PENAL INST', 'CAUSE/BRG CON SUB PENAL INST', 'CONCEAL HOMICIDAL DEATH', 'CONCEAL/AID FUGITIVE', 'CONSP AGAINST CIVIL RIGHTS', 'DISARM PC OFF/CORR INST EMP', 'DISARMING A PEACE OFFICER', 'ESCAPE FROM DEPT OF CORRECTION', 'ESCAPE OF FELON FROM PENAL INST', 'ESCAPE/VIOLATE ELEC MONITORING', 'FAIL REPORT ACCIDENT/DEATH/INJURY', 'FAILURE REPORT ACCIDENT/DEATH', 'FAILURE REPORT ACCIDENT/INJURY', 'FALSE ALARM/COMPLAINT TO 911', 'FALSE PERSONATION OF POLICE', 'FALSE REPORT OF OFFENSE', 'FELON ESCAPE/PEACE OFFICER', 'FELON FAIL TO RETURN FROM FURLOUGH', 'FELON PROBATIONER ESCAPE OFF', 'FLEEING/ATTEMPT ELUDE OFF 3+', 'INDIRECT CRIMINAL CONTEMPT', 'MANUF/POSS ADULTERANTS DRUG TEST', 'MONEY LAUNDERING >$500,000', 'OBSTRUCT JUST/DESTROY EVIDENCE', 'OBSTRUCTING JUSTICE', 'POSS CONTROL SUBST PENAL INST', 'POSS CONTROL SUBSTANCE IN PENAL INST', 'POSS FRAUDULENT ID CARD', 'POSS/FRAUD/DL/PERMIT', 'POSSESS CANNIBIS PENAL INST', 'POSSESS ELEC CONTRABAND PENAL INST', 'POSSESS WEAPON IN PENAL INST', 'RESIST/OBSTRUC OFFICER/INJURY', 'RICO ACQUIRE ENTERPRISE', 'RICO CONSPIRACY TO VIOLATE', 'TERRORISM/FALSE THREAT', 'THREATEN PUBLIC OFFICIAL', 'THREATEN PUBLIC OFFICIAL/2ND', 'UUW-FELON POSS WEAPON IN PRISON', 'VENDOR FRAUD/KICKBACKS VIOL', 'CRIM DAMAGE/GOVT PROP/EXPL/<$500', 'CRIM DMG/GOVT PROP/<$500', 'CRIM DMG/GOVT PROP/>$500-$10K', 'RESIST/OBSTRUCT OFFICER/INJURY', 'UUW-FELON POSS/USE WEAP/PRISON', 'CRIM DAMAGE/STATE PROP/FIRE>$100,000', 'POSS/DISP ID CARD TO ANOTHER', 'ATT FOIL/DEFEAT SCREEN TEST', 'ATTEMPT ESCAPE FELON FROM PENAL INST', 'AGGRAVATED CRUELTY TO ANIMALS', 'ANIMAL TORTURE/2ND', 'CRUELTY TO ANIMALS/2ND', 'DOG FIGHTING/ATTEND SHOW']

df.loc[df['Holding Offense'].isin(inj_death_list), 'Holding Offense Cat'] = 'Person - Death/Physical Injury'
df.loc[df['Holding Offense'].isin(inj_death_unclear_list), 'Holding Offense Cat'] = 'Person - Death/Physical Injury Unclear'
df.loc[df['Holding Offense'].isin(sex_list), 'Holding Offense Cat'] = 'Sex'
df.loc[df['Holding Offense'].isin(drugs_list), 'Holding Offense Cat'] = 'Drugs'
df.loc[df['Holding Offense'].isin(property_list), 'Holding Offense Cat'] = 'Property'
df.loc[df['Holding Offense'].isin(weapon_list), 'Holding Offense Cat'] = 'Firearms/Weapon'
df.loc[df['Holding Offense'].isin(vehicle_list), 'Holding Offense Cat'] = 'DUI/Vehicle Violation Without Death/Physical Injury'
df.loc[df['Holding Offense'].isin(registry_list), 'Holding Offense Cat'] = 'Sex Offender/MVOAY Registry Violation'
df.loc[df['Holding Offense'].isin(other_list), 'Holding Offense Cat'] = 'State/Other'


# categorize unlisted offenses based on "like" criteria
# ----- these won't be 100% accurately categorized, but it should be close -----
offense_cat_dict = {'SEX OFF': 'Sex Offender/MVOAY Registry Violation',
                    'SEX': 'Sex',
                    'INDEC': 'Sex',
                    'MURDER': 'Person - Death/Physical Injury',
                    'HOMIC': 'Person - Death/Physical Injury',
                    'HARM': 'Person - Death/Physical Injury',
                    'PORN': 'Sex',
                    'DUI': 'DUI/Vehicle Violation Without Death/Physical Injury',
                    'ASSAULT': 'Person - Death/Physical Injury Unclear',
                    'BATTERY': 'Person - Death/Physical Injury Unclear',
                    'DISCH': 'Firearms/Weapon',
                    'FIREA': 'Firearms/Weapon',
                    'THEFT': 'Property',
                    'STOLE': 'Property',
                    'METH': 'Drugs',
                    'KINAP': 'Person - Death/Physical Injury Unclear',
                    'ROBB': 'Property',
                    'WEAP': 'Firearms/Weapon',
                    'GUN': 'Firearms/Weapon',
                    'POSS': 'Drugs',   # this one especially contains non-drug offenses
                    'ARSON': 'Property',
                    'ABD': 'Person - Death/Physical Injury Unclear',
                    'DEL': 'Drugs',
                    'MANS': 'Person - Death/Physical Injury',
                    'MAN': 'Drugs',
                    'ENDANGER': 'Person - Death/Physical Injury Unclear',
                    'CANNA': 'Drugs',
                    'PROP': 'Property',
                    'DRUG': 'Drugs'
                    }

# for each key word, label all holding offenses that contain that key word
for key, val in offense_cat_dict.items():    
    df.loc[(df['Holding Offense'].str.contains(key)) & (pd.isnull(df['Holding Offense Cat'])), 'Holding Offense Cat'] = val

    
# label those that are still uncategorized as other
df.loc[pd.isnull(df['Holding Offense Cat']), 'Holding Offense Cat'] = 'State/Other'


if len(df) != len1:
    print('ERROR: categorizing offenses changed the df length')
    1/0
    
print(df['Holding Offense Cat'].value_counts(dropna=False))



In [ ]:
# prep for sankey diagram
holding_sankey_df = df[['Holding Offense Cat', 'Offense Type', 'IDOC #', 'Current Admission Date']].drop_duplicates()
holding_sankey_df = holding_sankey_df[['Holding Offense Cat', 'Offense Type', 'IDOC #']].groupby(['Holding Offense Cat', 'Offense Type']).count().reset_index().rename(columns={'IDOC #': 'Count'})
print(holding_sankey_df)

# get the node df - a df dictionary of admission types and indices
node_df = pd.concat([holding_sankey_df[['Holding Offense Cat']].rename(columns={'Holding Offense Cat': 'AT'}),
                       holding_sankey_df[['Offense Type']].rename(columns={'Offense Type': 'AT'})])
node_df = node_df.drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index': 'ID'})
node_df['Color'] = get_hex(len(node_df))

# get the links df
links_df = pd.merge(holding_sankey_df, 
                    node_df[['AT', 'ID', 'Color']].rename(columns={'ID': 'Source', 'Color': 'Link Color'}), 
                    left_on='Holding Offense Cat', right_on='AT', how='inner')
links_df = pd.merge(links_df, 
                    node_df[['AT', 'ID']].rename(columns={'ID': 'Target'}), 
                    left_on='Offense Type', right_on='AT', how='inner')
links_df['Link Color'] = links_df['Link Color'].apply(lambda x: 'rgba' + str(get_rgb_from_hex(x) + (0.5,)))
links_df = links_df[['Source', 'Target', 'Link Color', 'Count']]

# create the chart
fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 20,
        thickness = 30,
        line = dict(
            color = 'black',
            width = 0
        ),
        label = node_df['AT'],
        color = node_df['Color']
    ),
    link = dict(
        source = links_df['Source'],
        target = links_df['Target'],
        value = links_df['Count'],
        color = links_df['Link Color']
    )
)])

fig.update_layout(title_text="Difference between Phil's Categories and IDOC Offense Type", 
                  height=1000, 
                  font_size=12)
fig.show()
                                                                                                  


Phil's categories match pretty well with IDOC's.

**Recidivism calculation**

IDOC changed the way that it records recidivism over the years. Recalculate recidivism as sentence number.

In [ ]:
# convert admission date to a datetime
df['Current Admission Date'] = pd.to_datetime(df['Current Admission Date'])

len1 = len(df)

# rank the admission date per person - this is the nth time they've been admitted
df_admis_rank = df[['IDOC #', 'Current Admission Date']].drop_duplicates()
df_admis_rank['Sentence Number'] = df_admis_rank.groupby(['IDOC #'])['Current Admission Date'].rank(ascending=True)

# join back to the full dataframe
df = pd.merge(df, df_admis_rank, on=['IDOC #', 'Current Admission Date'], how='inner')

if len(df) != len1:
    print('ERROR: Adding the sentence number changed the df length')



**Eligibility for Sentencing Credits**

For the purposes of our analysis, someone is *ineligible* for earned discretionary sentencing credits (EDSC) if:
* Their holding offense is related to gunrunning or sex
* They're sentenced to life
* They're required to serve at least 85% of their sentence under truth-in-sentencing
* They're considered a sexually dangerous person (SDP)

Otherwise, they are eligible.

In [ ]:
df['EDSC Eligible'] = None
df.loc[(((df['TIS Pct'] >= 0.85) & (~pd.isnull(df['TIS Pct'])))
      | (df['Holding Offense'].str.contains('GUNRUNNING'))
      | (df['Sentence Years'].isin(['LIFE', 'SDP']))
      | (df['Holding Offense Cat'] == 'Sex')),
      'EDSC Eligible'] = 'N'

df.loc[pd.isnull(df['EDSC Eligible']), 'EDSC Eligible'] = 'Y'

print(df['EDSC Eligible'].value_counts())



**Exporting the data**

The full data set is 1.8 million records. Export with understanding.

In [ ]:
print(len(df))

In [ ]:
# export more recent data
# df_recent = df[df['Doc Date'] >= date(2014, 1, 1)]
# print(len(df_recent))
# df_recent.to_csv('pop_data_2014_thru_20220331.csv')

# # export single institution all time
# df_export = df[df['Parent Institution'] == 'Logan']
# print(len(df_export))


In [ ]:
# df_recent.to_csv('pop_data_20140101-20211231.csv')
df.to_csv('pop_data_thru_20220930.csv')

In [ ]:
tmp_df = df[(df['Doc Date'].apply(lambda x: x.date()) >= date(2022, 2, 1))] #& (df['Doc Date'] <= date(2018, 9, 30))]
print(len(tmp_df))

In [ ]:
tmp_df.to_csv('tmp_df.csv')